## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [0]:
%fs ls /FileStore/tables/

path,name,size
dbfs:/FileStore/tables/Bookings.csv,Bookings.csv,170118
dbfs:/FileStore/tables/Facilities.csv,Facilities.csv,494
dbfs:/FileStore/tables/Members.csv,Members.csv,3313
dbfs:/FileStore/tables/employees0.json,employees0.json,450


In [0]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [0]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
-- bookid: integer (nullable = true)
-- facid: integer (nullable = true)
-- memid: integer (nullable = true)
-- starttime: string (nullable = true)
-- slots: integer (nullable = true)

Facilities Schema
root
-- facid: integer (nullable = true)
-- name: string (nullable = true)
-- membercost: double (nullable = true)
-- guestcost: double (nullable = true)
-- initialoutlay: integer (nullable = true)
-- monthlymaintenance: integer (nullable = true)

Members Schema
root
-- memid: integer (nullable = true)
-- surname: string (nullable = true)
-- firstname: string (nullable = true)
-- address: string (nullable = true)
-- zipcode: integer (nullable = true)
-- telephone: string (nullable = true)
-- recommendedby: integer (nullable = true)
-- joindate: string (nullable = true)

### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [0]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [0]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [0]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [0]:
%sql
select * from facilities where membercost > 0

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5.0,25.0,10000,200
1,Tennis Court 2,5.0,25.0,8000,200
4,Massage Room 1,9.9,80.0,4000,3000
5,Massage Room 2,9.9,80.0,4000,3000
6,Squash Court,3.5,17.5,5000,80


####  Q2: How many facilities do not charge a fee to members?

In [0]:
%sql
select count(*) as count_facilities_do_not_charge from facilities where membercost = 0

count_facilities_do_not_charge
4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [0]:
%sql
select * from facilities where membercost < monthlymaintenance*0.2

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5.0,25.0,10000,200
1,Tennis Court 2,5.0,25.0,8000,200
2,Badminton Court,0.0,15.5,4000,50
3,Table Tennis,0.0,5.0,320,10
4,Massage Room 1,9.9,80.0,4000,3000
5,Massage Room 2,9.9,80.0,4000,3000
6,Squash Court,3.5,17.5,5000,80
7,Snooker Table,0.0,5.0,450,15
8,Pool Table,0.0,5.0,400,15


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [0]:
%sql
select * from facilities where facid in (1,5)

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


In [0]:
display(facilities_df.filter("facid in (1,5)"))

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [0]:
%sql
select *, case when monthlymaintenance > 100 then 'expensive' else 'cheap' end as price_range from facilities

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance,price_range
0,Tennis Court 1,5.0,25.0,10000,200,expensive
1,Tennis Court 2,5.0,25.0,8000,200,expensive
2,Badminton Court,0.0,15.5,4000,50,cheap
3,Table Tennis,0.0,5.0,320,10,cheap
4,Massage Room 1,9.9,80.0,4000,3000,expensive
5,Massage Room 2,9.9,80.0,4000,3000,expensive
6,Squash Court,3.5,17.5,5000,80,cheap
7,Snooker Table,0.0,5.0,450,15,cheap
8,Pool Table,0.0,5.0,400,15,cheap


In [0]:
from pyspark.sql.functions import col, when
facilities =  spark.sql("select * from facilities")
facilities_cheap_exp = facilities.withColumn('price_range', when(facilities.monthlymaintenance > 100, 'expensive').otherwise('cheap'))
display(facilities_cheap_exp)

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance,price_range
0,Tennis Court 1,5.0,25.0,10000,200,expensive
1,Tennis Court 2,5.0,25.0,8000,200,expensive
2,Badminton Court,0.0,15.5,4000,50,cheap
3,Table Tennis,0.0,5.0,320,10,cheap
4,Massage Room 1,9.9,80.0,4000,3000,expensive
5,Massage Room 2,9.9,80.0,4000,3000,expensive
6,Squash Court,3.5,17.5,5000,80,cheap
7,Snooker Table,0.0,5.0,450,15,cheap
8,Pool Table,0.0,5.0,400,15,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [0]:
%sql
select firstname, surname from members where joindate = (select max(joindate) from members)

firstname,surname
Darren,Smith


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [0]:
%sql
select distinct f.name as court, concat(firstname, ' ', surname) as member from bookings b join facilities f on b.facid = f.facid and f.name like 'Tennis%' join members m on m.memid = b.memid order by member

court,member
Tennis Court 2,Anne Baker
Tennis Court 1,Anne Baker
Tennis Court 2,Burton Tracy
Tennis Court 1,Burton Tracy
Tennis Court 1,Charles Owen
Tennis Court 2,Charles Owen
Tennis Court 2,Darren Smith
Tennis Court 2,David Farrell
Tennis Court 1,David Farrell
Tennis Court 2,David Jones


In [0]:
from pyspark.sql.functions import col, when, concat
tennis = facilities_df.filter(col("name").like('Tennis%'))
members_tennis = bookings_df.join(tennis, tennis.facid==bookings_df.facid) \
  .join(members_df, members_df.memid==bookings_df.memid) \
  .withColumn("member", concat("firstname", "surname")).select("name", "member").drop_duplicates()
display(members_tennis)

name,member
Tennis Court 1,JohnHunt
Tennis Court 2,TimRownam
Tennis Court 2,JackSmith
Tennis Court 1,DavidJones
Tennis Court 2,NancyDare
Tennis Court 2,TracySmith
Tennis Court 1,CharlesOwen
Tennis Court 1,PonderStibbons
Tennis Court 1,FlorenceBader
Tennis Court 1,RamnareshSarwin


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [0]:
%sql
select f.name, concat(firstname, ' ', surname) as member, b.slots*2*(case b.memid when 0 then guestcost else membercost end) as charge, b.bookid
from bookings b join members m on b.memid=m.memid join facilities f on f.facid=b.facid and b.slots*2*(case when b.memid = 0 then guestcost else membercost end) > 30
where to_date(starttime) = '2012-09-14' order by charge desc

name,member,charge,bookid
Massage Room 2,GUEST GUEST,640.0,2946
Massage Room 1,GUEST GUEST,320.0,2940
Massage Room 1,GUEST GUEST,320.0,2937
Massage Room 1,GUEST GUEST,320.0,2942
Tennis Court 2,GUEST GUEST,300.0,2926
Tennis Court 1,GUEST GUEST,150.0,2920
Tennis Court 1,GUEST GUEST,150.0,2922
Tennis Court 2,GUEST GUEST,150.0,2925
Squash Court,GUEST GUEST,140.0,2948
Massage Room 1,Jemima Farrell,79.2,2941


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [0]:
%sql
select * from (
  select f.name, concat(firstname, ' ', surname) as member, b.bookid --, membercost, guestcost, slots
  , case when m.memid = 0 then guestcost * slots * 2 else membercost*slots*2 end as charge    -- cost listed in facilities are per half-hour slot.  Thus the cost is slots*2 (for 1 hour) * _cost
  from bookings b join facilities f on b.facid=f.facid join members m on b.memid=m.memid
  where to_date(starttime) = '2012-09-14' ) a
where charge > 30 order by charge desc

name,member,bookid,charge
Massage Room 2,GUEST GUEST,2946,640.0
Massage Room 1,GUEST GUEST,2940,320.0
Massage Room 1,GUEST GUEST,2937,320.0
Massage Room 1,GUEST GUEST,2942,320.0
Tennis Court 2,GUEST GUEST,2926,300.0
Tennis Court 1,GUEST GUEST,2920,150.0
Tennis Court 1,GUEST GUEST,2922,150.0
Tennis Court 2,GUEST GUEST,2925,150.0
Squash Court,GUEST GUEST,2948,140.0
Massage Room 1,Jemima Farrell,2941,79.2


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [0]:
%sql
select name, round(sum(charge),2) as revenue from (
  select f.facid, f.name, concat(firstname, ' ', surname) as member, b.bookid 
  , case when m.memid = 0 then guestcost * slots * 2 else membercost*slots*2 end as charge
  from bookings b join facilities f on b.facid=f.facid join members m on b.memid=m.memid
) b group by name having revenue < 1000 order by revenue

name,revenue
Table Tennis,360.0
Snooker Table,480.0
Pool Table,540.0
